In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time
import re

In [5]:
train_data = pd.read_csv("Data\snli_tr_train.csv",encoding="utf-8")
test_data = pd.read_csv("Data\snli_tr_test.csv",encoding="utf-8")

print(train_data.head())
print(test_data.head())

   idx                                            premise  \
0    0  Attaki bir kişi, bozuk bir uçağın üzerinden at...   
1    1  Attaki bir kişi, bozuk bir uçağın üzerinden at...   
2    2  Attaki bir kişi, bozuk bir uçağın üzerinden at...   
3    3  Fotoğraf makinesinde gülümseyen ve sallayan ço...   
4    4  Fotoğraf makinesinde gülümseyen ve sallayan ço...   

                                     hypothesis  label  
0         Bir kişi atını yarışma için eğitiyor.      1  
1  Bir kişi bir lokantada omlet sipariş ediyor.      2  
2            Bir kişi açık havada, at üzerinde.      0  
3                     Ailelerine gülümsüyorlar.      1  
4                          Burada çocuklar var.      0  
   idx                                            premise  \
0    0  Bu kilise korosu, kilisedeki kitaptan neşeli ş...   
1    1  Bu kilise korosu, kilisedeki kitaptan neşeli ş...   
2    2  Bu kilise korosu, kilisedeki kitaptan neşeli ş...   
3    3  Yeşil başörtüsü, mavi gömlekli ve çok b

In [4]:
# train_data.drop("idx",inplace=True,axis=1)
# test_data.drop("idx",inplace=True,axis=1)
# print(train_data.head())
# print(test_data.head())

                                             premise  \
0  Attaki bir kişi, bozuk bir uçağın üzerinden at...   
1  Attaki bir kişi, bozuk bir uçağın üzerinden at...   
2  Attaki bir kişi, bozuk bir uçağın üzerinden at...   
3  Fotoğraf makinesinde gülümseyen ve sallayan ço...   
4  Fotoğraf makinesinde gülümseyen ve sallayan ço...   

                                     hypothesis  label  
0         Bir kişi atını yarışma için eğitiyor.      1  
1  Bir kişi bir lokantada omlet sipariş ediyor.      2  
2            Bir kişi açık havada, at üzerinde.      0  
3                     Ailelerine gülümsüyorlar.      1  
4                          Burada çocuklar var.      0  
                                             premise  \
0  Bu kilise korosu, kilisedeki kitaptan neşeli ş...   
1  Bu kilise korosu, kilisedeki kitaptan neşeli ş...   
2  Bu kilise korosu, kilisedeki kitaptan neşeli ş...   
3  Yeşil başörtüsü, mavi gömlekli ve çok büyük bi...   
4  Yeşil başörtüsü, mavi gömlekli ve çok 

In [6]:
train_data["label"].value_counts()

label
 0    183416
 2    183187
 1    182764
-1       785
Name: count, dtype: int64

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550152 entries, 0 to 550151
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   idx         550152 non-null  int64 
 1   premise     550152 non-null  object
 2   hypothesis  550152 non-null  object
 3   label       550152 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 16.8+ MB


In [8]:
cleanedData = []

#nltk.download('stopwords')
swords = stopwords.words('turkish')

def preprocess_text(sen):
    #sayıları silme
    sentence = re.sub('[\d\s]',' ',str(sen))
    #noktalama işaretlerini silme
    sentence = re.sub('[^\w\s]',' ',str(sentence))
    #birden çok boşluk silme
    sentence = re.sub(r'\s+',' ',sentence)
    #tek karakterleri silme
    sentence = re.sub(r"\b[\w\w]\b",' ',str(sentence))

    #engellenecek kelimeleri silme
    WPT = nltk.WordPunctTokenizer()
    tokens = WPT.tokenize(sentence)
    filtered_tokens = [token for token in tokens if token not in swords]
    single_doc = ' '.join(filtered_tokens)

    #hüçük harf dönüştürme
    return single_doc.lower()

x = train_data['premise']
y = train_data['hypothesis']
z = train_data['label']

a = test_data['premise']
b = test_data['hypothesis']
c = train_data['label']

x = x.apply(preprocess_text)
y = y.apply(preprocess_text)

a = a.apply(preprocess_text)
b = b.apply(preprocess_text)

print(x.head())
print(a.head())


0     attaki bir kişi bozuk bir uçağın üzerinden atlar
1     attaki bir kişi bozuk bir uçağın üzerinden atlar
2     attaki bir kişi bozuk bir uçağın üzerinden atlar
3    fotoğraf makinesinde gülümseyen sallayan çocuklar
4    fotoğraf makinesinde gülümseyen sallayan çocuklar
Name: premise, dtype: object
0    bu kilise korosu kilisedeki kitaptan neşeli şa...
1    bu kilise korosu kilisedeki kitaptan neşeli şa...
2    bu kilise korosu kilisedeki kitaptan neşeli şa...
3    yeşil başörtüsü mavi gömlekli büyük bir sırıtı...
4    yeşil başörtüsü mavi gömlekli büyük bir sırıtı...
Name: premise, dtype: object


In [9]:
vectorizer = CountVectorizer(max_features=10000)
BOW = vectorizer.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(BOW,np.asarray(train_data["label"]))

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(412614, 10000)
(137538, 10000)
(412614,)
(137538,)


In [19]:
from sklearn.svm import SVC
start_time = time.time()

model = SVC()
model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting SVC took {} seconds".format(process_time))

In [ ]:
predictions = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))